This notebook is meant to be a simpler(shorter) version of [programming manual](https://docs.modular.com/mojo/programming-manual.html)
created during going over it.\
This notebook will cover `Mojo compiler` and `Basic systems programming extensions`.

## Mojo compiler
You can run Mojo program from a terminal, just like you can with Python.
just type:
 > mojo `filename.mojo`
```bash
$ cat hello.🔥
def main():
    print("hello world")
    for x in range(9, 0, -3):
        print(x)
$ mojo hello.🔥
hello world
9
6
3
$
```


# Basic systems programming extensions

Python's support for systems programming is mainly delegated to C, and we want to a single system that is great in that world. \
As such, mojo implants those components and features.

### `let` and `var` declarations

- System programmers often want to declare that a value is immutable for type-safety and performance
- They may want to get an error if they mistype a variable name in an assignment.

Mojo provides scoped runtime value declaration:
- `let` is immutable
- `var` is mutable

In [ ]:
def your_function(a, b):
    let c = a
    # Uncomment to see an error:
    # c = b  # error: c is immutable

    if c != b:
        let d = b
        print(d)

your_function(2, 3)

3


`let` and `var` declarations support type specifiers as well as patterns, and late initialization:

In [ ]:
def your_function():
    let x: Int = 42
    let y: Float64 = 17.0

    let z: Float32
    if x != 0:
        z = 1.0
    else:
        z = foo()
    print(x,y,z)

def foo() -> Float32:
    return 3.14

your_function()

42 17.0 1.0


Note that `let` and `var` are optional when in a def function, but they are required for all variables in an `fn` function

Also beware that when using Mojo in a REPL environment, top-level variables are treaded like variables in a `def`, so they allow implicit value type declarations.

### `struct` types

An important feature of modern systems programming languages is the ability to build high-level and safe abstractions on top of these complex, low-level operations without any performance loss. In Mojo, this is provided by the `struct` type.

A `struct` in Mojo is similar to Python `class`. Their differences are as follows:

- Python classes are dynamic: they allow for dynamic dispatch, monkey-patching, and dynamically binding instance properties at runtime.
- Mojo struct are static: they are bound at compile-time. Structs allow you to trade flexibility for performance while being safe and easy to use.

Here is a simple `struct`:

In [ ]:
struct MyPair:
    var first: Int
    var second: Int

    # We use 'fn' instead of 'def' here - we'll explain that soon
    fn __init__(inout self, first: Int, second: Int):
        self.first = first
        self.second = second

    # __lt__  -> less than `<`
    fn __lt__(self, rhs: MyPair) -> Bool:
        return self.first < rhs.first or
              (self.first == rhs.first and
               self.second < rhs.second)

Syntactically, the biggest difference compered to a Python `class` is that all instances of properties in a `struct` **must** be explicitly declared with a `var` or `let` declaration.

In Mojo, the structure and contents of a `struct` are set in advance and can't be changed while the program is running. \
Unlike in Python, where you can add, remove, or change attributes of an object on the fly. Mojo doesn't allow that for structs.\
This means you can't use `del` to remove methods or change its value in the middle of running the program.

However, the static nature of `struct`, helps Mojo run your code faster. \
Mojo's structs also work really well with features you might already know from Python, like operator overloading. \
Furthermore, all the "standard types" are made using structs.


## `Int` vs `int`
In Mojo, you might notice the use of `Int`, which is different from Python's `int`.
In Python, the `int` type can handle huge number and has some extra features, like checking if two number are the same object.
But this comes with some extra baggage that can slow thing down.

In Mojo, the `Int` is different. It's designed to be simple, fast, and tuned for your computer's hardware to handle quickly.

- Allows programmers who need to work closely with computer hardware a transparent and reliable way to interact with hardware. We don't want to rely on fancy tricks (like JIT compilers) to make things faster.
- Allows Mojo to work well with Python without causing any issues. By using a different name (`Int` instead of `int`), we can keep both types in Mojo without changing how Python's ints work.


As a bonus, `Int` follows the same naming style as other custom data types you might create in Mojo. Additionally, `Int` is `struct` that's included in Mojo's standard set of tools.

## Strong type checking

Even though you can still use flexible types like in Python, Mojo lets you use strict type checking. Type-checking can make your code more predictable, manageable, and secure.

One of the ways to employ type checking is to use `struct` types. \
For example:

In [ ]:
def pair_test() -> Bool:
    let p = MyPair(1, 2)
    let b = MyPair(3, 4)
    # Uncomment to see an error:
    #return p < 4 # gives a compile time error
    return p < b

print(pair_test())

True


If you uncomment the first return statement and run it, you'll get a compile-time error telling you that `4` cannot be converted to a `MyPair`. \
Type checking isn't the only use-case for strong types. \
Since we know the types are accurate, we can optimize the code based on those types, pass values in register, and be as efficient as `C` for argument passing and other low-level details.


## Overloading functions and methods

Like in Python, you can define functions in Mojo without specifying argument data types and Mojo will handle them dynamically.\
However, when you want to ensure type safety, Mojo also offers full support for overloaded functions and methods.

This allows you to define multiple versions of a function or method that have the same name but different argument types. \

When resolving a function call, Mojo tries each candidate and uses the one that works, or it picks the closest match, \ 
or it reports that the call is ambiguous if it can't figure out which one to use.

Here is an example:

In [ ]:
struct Complex:
    var re: Float32 # real part
    var im: Float32 # imagainary part

    fn __init__(inout self, re: Float32):
        # Constructs a complex number with no imaginary part
        self.re = re
        self.im = 0
    
    fn __init__(inout self, r: Float32, i: Float32):
        # Constructs a complex number with both real and imaginary parts
        self.re = r
        self.im = i

You can overload methods in structs and classes and overload module-level functions.

If you leave your argument names without type definitions, then the function behaves just like Python. \
As soon as you define a single argument type, Mojo will look for overload candidates and resolve function calls.

## `fn` definitions

To recap, `def` is defined by necessity to be very dynamic, flexible and generally compatible with Python. \
Mojo provides an `fn` declaration, which is like a "strict mode" for `def`.

As far as caller is concerned, `fn` and `def` are interchangeable: there is nothing a `def` can provide that `fn` can't (and vice versa). \
Although `fn`'s have a number of limitations compared to `def` functions:
- Argument values default to being immutable in the body of the function, instead of mutable. This catches accidental mutations, and permits the use of non-copyable types as arguments
- Argument values require a type specification. Similarly, a missing return type specifier is interpreted as returning None instead of an unknown return type. Note that both can be explicitly declared to return `object`, which allows one to opt-in to the behavior of `def` functions.
- Implicit declaration of local variables is disabled, so all locals must be declared. This catches typos and dovetails.
- Both support raising exceptions, but this must explicitly declared on a `fn` with the `raises` keyword.


## The `__copyinit__`, `__moveinit__`, and `__takeinit__` special methods.

Mojo supports full "value semantics" as seen in C++, and it makes defining simple aggregates of fields very easy with the @value decorator. \

Mojo allows you to define custom constructors, custom destructors, and custom copy constructors using the `__init__`, `__copyinit__`, `__moveinit__`, and `__takeinit__` special methods.

For example, consider a dynamic string type that needs to allocate memory for the string data when constructed and destroy it when the value is destroyed : 

In [ ]:
from memory.unsafe import Pointer

struct HeapArray:
    var data: Pointer[Int]
    var size: Int

    fn __init__(inout self, size: Int, val: Int):
        self.size = size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i,val)

    fn __del__(owned self):
        print("Freeing array of size ", self.size)
        self.data.free()
    
    fn dump(self):
        print_no_newline("[") # try normal print
        for i in range(self.size):
            if i > 0:
                print_no_newline(", ")
            print_no_newline(self.data.load(i))
        print("]")


The array type is implemented using low level functions to show a simple example of how this works. However, if you try to copy an instance of `HeapArray` with `=` operator, you might be surprised ...

In [ ]:
var a = HeapArray(10, 42)
a.dump() # Will print [42, 42, 42, 42, 42, 42, 42, 42, 42, 42]
# Uncomment to see an error:
# var b = a # We get a ERROR: Because we have not defined __copyinit__

var b = HeapArray(10, 0)
b.dump()
a.dump()

[42, 42, 42, 42, 42, 42, 42, 42, 42, 42]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[42, 42, 42, 42, 42, 42, 42, 42, 42, 42]


To enable copying, you need to define a `__copyinit__` method that takes a `HeapArray` as an argument and copies the data from the other array into the new array. \

In [ ]:
struct HeapArray:
    var data: Pointer[Int]
    var size: Int

    fn __init__(inout self, size: Int, val: Int):
        self.size = size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, val)

    fn __copyinit__(inout self, other: HeapArray):
        self.size = other.size
        self.data = Pointer[Int].alloc(self.size)
        for i in range(self.size):
            self.data.store(i, other.data.load(i))

    fn __del__(owned self):
        self.data.free()

    fn dump(self):
        print_no_newline("[")
        for i in range(self.size):
            if i > 0:
                print_no_newline(", ")
            print_no_newline(self.data.load(i))
        print("]")

With this implementation, our code should work as expected and cope with the `=` operator.

In [ ]:
var a = HeapArray(10,42)
var b = a
print_no_newline("a: ")
a.dump()
print_no_newline("b: ")
b.dump()

a: [42, 42, 42, 42, 42, 42, 42, 42, 42, 42]
b: [42, 42, 42, 42, 42, 42, 42, 42, 42, 42]


Mojo also supports the `__moveinit__` method which transfers a value from one place to another when the source lifetime ends, and the -  
`__takeinit__` method where the contents of a value is logically transferred out of the source, but its destructor is still run, and allows defining custom move logic.

'''
There the chapter ends, you can see the [programming manual](https://docs.modular.com/mojo/programming-manual.html) for more details.
or next chapter in another notebook.
'''